In [1]:
from clickzetta.zettapark.session import Session
from clickzetta.zettapark.functions import unix_timestamp, col, current_timestamp, from_unixtime

In [2]:
import json
# 从配置文件中读取参数
with open('security/config-uat.json', 'r') as config_file:
    config = json.load(config_file)

# 创建会话
session = Session.builder.configs(config).create()

In [4]:
inputData = [("2019-07-01 12:01:19",
            "07-01-2019 12:01:19", 
            "07-01-2019")]
columns=["timestamp_1","timestamp_2","timestamp_3"]
df=session.createDataFrame(
        data = inputData,
        schema = columns)
df.printSchema()
df.show()

root
 |-- `timestamp_1`: StringType() (nullable = False)
 |-- `timestamp_2`: StringType() (nullable = False)
 |-- `timestamp_3`: StringType() (nullable = False)
-----------------------------------------------------------
|timestamp_1          |timestamp_2          |timestamp_3  |
-----------------------------------------------------------
|2019-07-01 12:01:19  |07-01-2019 12:01:19  |07-01-2019   |
-----------------------------------------------------------



In [5]:
df2 = df.select( 
      unix_timestamp(col("timestamp_1")).alias("timestamp_1"), 
      unix_timestamp(col("timestamp_2"),"MM-dd-yyyy HH:mm:ss").alias("timestamp_2"), 
      unix_timestamp(col("timestamp_3"),"MM-dd-yyyy").alias("timestamp_3"), 
      unix_timestamp(current_timestamp()).alias("timestamp_4") # 使用当前时间的 Unix 时间戳
   )
df2.printSchema()
df2.show()

df3=df2.select(
    from_unixtime(col("timestamp_1")).alias("timestamp_1"),
    from_unixtime(col("timestamp_2"),"MM-dd-yyyy HH:mm:ss").alias("timestamp_2"),
    from_unixtime(col("timestamp_3"),"MM-dd-yyyy").alias("timestamp_3"),
    from_unixtime(col("timestamp_4")).alias("timestamp_4")
  )
df3.printSchema()
df3.show()

root
 |-- `timestamp_1`: LongType() (nullable = True)
 |-- `timestamp_2`: LongType() (nullable = True)
 |-- `timestamp_3`: LongType() (nullable = True)
 |-- `timestamp_4`: LongType() (nullable = True)
---------------------------------------------------------
|timestamp_1  |timestamp_2  |timestamp_3  |timestamp_4  |
---------------------------------------------------------
|1561953679   |1561953679   |1561910400   |1735025999   |
---------------------------------------------------------

root
 |-- `timestamp_1`: TimestampType(tz=ltz) (nullable = True)
 |-- `timestamp_2`: TimestampType(tz=ltz) (nullable = True)
 |-- `timestamp_3`: TimestampType(tz=ltz) (nullable = True)
 |-- `timestamp_4`: TimestampType(tz=ltz) (nullable = True)
-------------------------------------------------------------------------------------
|timestamp_1                |timestamp_2  |timestamp_3  |timestamp_4                |
-------------------------------------------------------------------------------------
|2019

In [6]:
session.close()